# Preprocessing
H. K. Kim, Y. Kim, S. Lee, S. Min, J. Y. Bae, J. W. Choi, J. Park, D. Jung, S. Yoon, and H. H. Kim. (2019). Spcas9 activity prediction by deepspcas9, a deep learning–based model with high generalization performance. Science Advances, 5(11), eaax9249.

This file was used to preprocess the data obtained from supplementary tables 1 and 2.

In [1]:
import pandas as pd
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
import numpy as np
import torch as t

In [2]:
def encode(seq, z='ATCG'):
    return [list(map(lambda x: 1 if x==c else 0, z)) for c in seq]
assert(encode('ATCG') == [[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]])

In [3]:
kimData_train = pd.read_excel("kim_2019_table_s1.xlsx")
kimData_test = pd.read_excel("kim_2019_table_s2.xlsx")
kimData_test.rename({"Background subtracted indel frequencies\n(average, %)" : "Background subtracted indel (%)"}, axis=1, inplace=True)
kimData_train = kimData_train[["Target context sequence (4+20+3+3)", "Background subtracted indel (%)"]]
kimData_test = kimData_test[["Target context sequence (4+20+3+3)", "Background subtracted indel (%)"]]
kimData = pd.concat((kimData_train, kimData_test))
kimData.drop_duplicates("Target context sequence (4+20+3+3)", inplace=True)
kimData.rename({"Target context sequence (4+20+3+3)" : "seq", "Background subtracted indel (%)": "Indel freq"}, axis=1, inplace=True)

In [4]:
onehotEncoded = []

for seq in kimData["seq"]:
    onehotEncoded.append(np.array(encode(seq)).transpose().tolist())

kimData.insert(1, "Onehot Encoding", onehotEncoded)
kimData.head()

,seq,Onehot Encoding,Indel freq
0,TTCTGCCTTGTTTCTTTCCTCTCTGGGTCG,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",24.287805
1,ACGACCTTCAGCTCAGTGACAGTGAGGACA,"[[1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,...",69.500438
2,AGGACGACGACTACAATAAGCCTCTGGATC,"[[1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,...",25.994760
3,GCAGCAAACTGACGGAGAACCTTGTGGCCC,"[[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,...",57.964590
4,CCGGCAGATATCCGTGAAGGCTCTAGGTAC,"[[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,...",39.355020


In [5]:
meltingTemp = []

for seq in kimData["seq"]:
    myseq = Seq(seq)
    meltingTemp.append(mt.Tm_NN(myseq))

kimData.insert(2, "Melting Point", meltingTemp)
kimData.head()

,seq,Onehot Encoding,Melting Point,Indel freq
0,TTCTGCCTTGTTTCTTTCCTCTCTGGGTCG,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",61.970567,24.287805
1,ACGACCTTCAGCTCAGTGACAGTGAGGACA,"[[1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,...",63.961100,69.500438
2,AGGACGACGACTACAATAAGCCTCTGGATC,"[[1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,...",60.971936,25.994760
3,GCAGCAAACTGACGGAGAACCTTGTGGCCC,"[[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,...",66.593277,57.964590
4,CCGGCAGATATCCGTGAAGGCTCTAGGTAC,"[[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,...",62.893776,39.355020


In [6]:
onehot = []
response = []
meltingpoint = []

for rowIdx, row in kimData.iterrows():
    onehot.append(row["Onehot Encoding"])
    response.append(float(row["Indel freq"]))
    meltingpoint.append(float(row["Melting Point"]))

_onehot = t.tensor(onehot, dtype=t.float32)
_response = t.tensor(response, dtype=t.float32)
_meltingpoint = t.tensor(meltingpoint, dtype=t.float32)

t.save(_onehot, f'kim_2019_X.pt')
t.save(_response, f'kim_2019_Y.pt')
t.save(_meltingpoint, f'kim_2019_Features.pt')

print(_onehot.shape, _response.shape, _meltingpoint.shape)

torch.Size([13359, 4, 30]) torch.Size([13359]) torch.Size([13359])
